In [1]:
import pandas as pd

In [2]:
f_orders = pd.read_csv("instacart/orders.csv", usecols = ["order_id", "user_id", "eval_set"])
f_order_product_train = pd.read_csv("instacart/order_products__prior.csv", usecols = ["order_id", "product_id"])
f_order_product_test = pd.read_csv("instacart/order_products__train.csv", usecols = ["order_id", "product_id"])

user_order = f_orders.groupby(["user_id"])["order_id"].apply(list)
order_product_train = f_order_product_train.groupby(["order_id"])["product_id"].apply(list)
order_product_test = f_order_product_test.groupby(["order_id"])["product_id"].apply(list)

order_product_list_order_train = order_product_train.keys().tolist()
order_product_list_product_train = order_product_train.values.tolist()

order_product_list_order_test = order_product_test.keys().tolist()
order_product_list_product_test = order_product_test.values.tolist()

data = f_orders.values.tolist()  #order_id, user_id, eval_set
train_data = list(filter(lambda x: x[2] == "prior", data))
test_data = list(filter(lambda x: x[2] == "train", data))

order_train_dict = dict()
for i, key in enumerate(order_product_list_order_train):
    order_train_dict[key] = order_product_list_product_train[i]

order_test_dict = dict()
for i, key in enumerate(order_product_list_order_test):
    order_test_dict[key] = order_product_list_product_test[i]


In [3]:
num_user = 200000

raw_train_dict = dict()
flag = 0
for line in train_data:
    key = line[1]
    order_id = line[0]
    if key not in raw_train_dict.keys():
        flag += 1
        if flag > num_user:
            break
        raw_train_dict[key] = [order_id]
    else:
        raw_train_dict[key].append(order_id)

filter_train_dict = dict()        
for (user, orders) in raw_train_dict.items():
    if len(orders) >= 10:
        filter_train_dict[user] = orders
        

test_dict = dict()
flag = 0
for line in test_data:
    key = line[1]
    order_id = line[0]
    if key not in test_dict.keys() and key in filter_train_dict.keys():
        test_dict[key] = []
        test_dict[key] += list(set(order_test_dict[order_id]))
    if key in test_dict.keys() and key in filter_train_dict.keys():
        test_dict[key] += list(set(order_test_dict[order_id]))



pre_train_dict = dict()
train_dict = dict()
train_bpr_dict = dict()
count = 0
for (user, orders) in filter_train_dict.items():
    if user in test_dict.keys():
        filter_orders = []
        for order in orders:
            if (len(order_train_dict[order])) > 2:
                filter_orders.append(order)
        if (len(filter_orders)) >=10:
            pre_train_dict[user] = [len(filter_orders[:-1])]
            train_bpr_dict[user] = []
            for order_id in filter_orders[:-1]:
                pre_train_dict[user] += list(set(order_train_dict[order_id]))
                train_bpr_dict[user] += list(set(order_train_dict[order_id]))
            train_dict[user] = list(set(order_train_dict[filter_orders[-3]]))
            train_dict[user] += list(set(order_train_dict[filter_orders[-2]]))
            train_dict[user] += list(set(order_train_dict[filter_orders[-1]]))
            train_bpr_dict[user] += list(set(order_train_dict[filter_orders[-1]]))

delete_key = []
for key in test_dict.keys():
    if key not in train_dict.keys():
        delete_key.append(key)

for key in delete_key:
    del test_dict[key]
        

ori_dict = dict() # user id starts from 0
flag = 0
for key in test_dict.keys():
    ori_dict[key] = flag
    flag += 1

new_pre_train_dict = dict((ori_dict[key], value) for (key, value) in pre_train_dict.items())
new_train_dict = dict((ori_dict[key], value) for (key, value) in train_dict.items())
new_test_dict = dict((ori_dict[key], value) for (key, value) in test_dict.items())
new_train_bpr_dict = dict((ori_dict[key], value) for (key, value) in train_bpr_dict.items())

print(len(train_dict.keys()))


56305


In [4]:
f_train = open("./Data/instacart/train.txt", "w")
for key in new_train_dict.keys():
    line = str(key)
    for product in new_train_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_train.write(line)
f_train.close()

f_train_bpr = open("./Data/instacart/train_bpr.txt", "w")
for key in new_train_bpr_dict.keys():
    line = str(key)
    for product in new_train_bpr_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_train_bpr.write(line)
f_train_bpr.close()

f_test = open("./Data/instacart/test.txt", "w")
for key in new_test_dict.keys():
    line = str(key)
    for product in new_test_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_test.write(line)
f_test.close()

In [5]:
f_train_mf = open("./Data/instacart/train_mf.txt","w")

for (u_id,ps) in new_pre_train_dict.items():
    n_order = ps[0]
    items = ps[1:]
    u_fre = dict()
    for p in items:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in u_fre.keys():
        fre = "{:.3f}".format(u_fre[p]/float(n_order))
        line = str(u_id)+ ' ' + str(p) + ' ' + str(fre) + '\n'
        f_train_mf.write(line)
f_train_mf.close()


In [6]:
f_fre_test = open("./Data/instacart/fre_test.txt","w")
for (u_id,ps) in new_pre_train_dict.items():
    n_order = ps[0]
    items = ps[1:]
    u_fre = dict()
    for p in items:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in new_train_dict[u_id]:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in u_fre.keys():
        fre = "{:.3f}".format(u_fre[p]/float(n_order+1))
        line = str(u_id)+ ' ' + str(p) + ' ' + str(fre) + '\n'
        f_fre_test.write(line)
f_fre_test.close()